In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data_path = '/kaggle/input/ift3395-ift6390-identification-maladies-retine/train_data.pkl'
test_data_path = '/kaggle/input/ift3395-ift6390-identification-maladies-retine/test_data.pkl'
# Load the pickle file
with open ( train_data_path, 'rb' ) as f :
    data = pickle.load (f)
# Access images and labels
images = data ['images']
images = np.array(images)
labels = data ['labels']
labels = np.array(labels)
print(images.shape)
#print(len(images[1][-1]))
#print(images[:2])
print(labels.shape)
#print(len(labels))
print(labels[:2])

In [ ]:
import torchvision.transforms as transforms
from PIL import Image
transform = transforms.ToPILImage()
image = transform(images[1])
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
flattened_images = images.reshape(images.shape[0], -1)
print(flattened_images.shape)

In [ ]:
np.random.seed(42)
num_samples = flattened_images.shape[0]

indices = np.arange(num_samples)
np.random.shuffle(indices)
shuffled_images = flattened_images[indices]
shuffled_labels = labels[indices]

baseline1Idxs = int(num_samples * 0.8)  # 80% for training, 20% for validation
print("baseline1Idxs: ", baseline1Idxs)
split_index=int(baseline1Idxs*0.8)
print("split_index: ", split_index)

train_images = shuffled_images[:split_index]
train_labels = shuffled_labels[:split_index]
print("train_images.shape: ", train_images.shape)
print("train_labels.shape: ", train_labels.shape)

val_images = shuffled_images[split_index:]
val_labels = shuffled_labels[split_index:]
print("val_images.shape: ", val_images.shape)
print("val_labels.shape: ", val_labels.shape)

In [ ]:
with open ( test_data_path , 'rb' ) as f:
    data1 = pickle.load (f)
# Access images and l a b e l s
testimages = data1 ['images']
testimages=np.array(testimages)
testimages.shape

In [ ]:
flattened_test_images = testimages.reshape(testimages.shape[0], -1)
print(flattened_test_images.shape)

In [ ]:
def make_original_array(preds):
        """
       
        """

        index_column = np.arange(1, preds.size + 1)

        # Reshape index_column and labels_train_modified to 2D column vectors
        index_column = index_column.reshape(-1, 1)
        labels_column = preds.reshape(-1, 1)

        # Concatenate the index column with the labels column
        combined_array = np.hstack((index_column, labels_column))

        return combined_array

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

np.random.seed(42)
# Initialize the RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf.fit(train_images, train_labels)

# Predict on validation set
val_preds = rf.predict(val_images)

# Calculate accuracy
val_accuracy = accuracy_score(val_labels, val_preds)
print(f'Validation Accuracy: {val_accuracy}')

# 
#Predict on test set
test_preds_rf = rf.predict(flattened_test_images)

# Convert predictions to original array format
predictions_rf = make_original_array(test_preds_rf)

# Create a DataFrame and save to CSV
df_rf = pd.DataFrame(predictions_rf, columns=['ID', 'label'])
df_rf.to_csv('submission.csv', index=False)